# GridSearchCV 

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from skorch import NeuralNetClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import numpy as np

# Define the neural network with two hidden layers
class TwoHiddenLayerNet(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim):
        super(TwoHiddenLayerNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.fc3 = nn.Linear(hidden_dim2, output_dim)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return self.softmax(x)

# Create synthetic dataset
X, y = make_classification(1000, 20, n_classes=2, random_state=42)

# Convert target labels to LongTensor
y = y.astype(np.int64)

# Define skorch NeuralNetClassifier
net = NeuralNetClassifier(
    TwoHiddenLayerNet,
    max_epochs=10,
    lr=0.1,
    optimizer=optim.SGD,
    criterion=nn.CrossEntropyLoss,
    batch_size=32,
    device='cuda' if torch.cuda.is_available() else 'cpu'  # Use GPU if available
)

# Define parameter grid for grid search
params = {
    'module__input_dim': [20],
    'module__hidden_dim1': [10, 50, 100],
    'module__hidden_dim2': [10, 50, 100],
    'module__output_dim': [2],
    'lr': [0.01, 0.1, 0.2],
    'optimizer': [optim.SGD, optim.Adam, optim.RMSprop],
    'max_epochs': [10, 20]
}

# Perform grid search with cross-validation
gs = GridSearchCV(net, params, refit=True, cv=3, scoring='accuracy', verbose=2, n_jobs=-1)
gs.fit(X.astype(np.float32), y)

# Print the best parameters and best score
print("Best parameters found: ", gs.best_params_)
print("Best cross-validation accuracy: ", gs.best_score_)

# Evaluate the best model on the whole dataset
y_pred = gs.predict(X.astype(np.float32))
accuracy = accuracy_score(y, y_pred)
print("Accuracy on the whole dataset: ", accuracy)


Fitting 3 folds for each of 162 candidates, totalling 486 fits
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6862       0.5700        0.6648  0.0405
      2        0.6372       0.8100        0.5976  0.0402
      3        0.5675       0.8550        0.5254  0.0375
      4        0.5121       0.8750        0.4823  0.0400
      5        0.4821       0.8900        0.4605  0.0385
      6        0.4661       0.8900        0.4486  0.0410
      7        0.4564       0.8950        0.4417  0.0400
      8        0.4501       0.8950        0.4373  0.0345
      9        0.4454       0.8950        0.4343  0.0390
     10        0.4418       0.8950        0.4322  0.0415
Best parameters found:  {'lr': 0.1, 'max_epochs': 10, 'module__hidden_dim1': 100, 'module__hidden_dim2': 10, 'module__input_dim': 20, 'module__output_dim': 2, 'optimizer': <class 'torch.optim.sgd.SGD'>}
Best cross-validation accuracy:  0.872977768187349

# RandomizedSearchCV

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from skorch import NeuralNetClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
from scipy.stats import uniform, randint
import numpy as np

# Define the neural network with two hidden layers
class TwoHiddenLayerNet(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim):
        super(TwoHiddenLayerNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim1)
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.fc3 = nn.Linear(hidden_dim2, output_dim)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return self.softmax(x)

# Create synthetic dataset
X, y = make_classification(1000, 20, n_classes=2, random_state=42)

# Convert target labels to LongTensor
y = y.astype(np.int64)

# Define skorch NeuralNetClassifier
net = NeuralNetClassifier(
    TwoHiddenLayerNet,
    max_epochs=10,
    lr=0.1,
    optimizer=optim.SGD,
    criterion=nn.CrossEntropyLoss,
    batch_size=32,
    device='cuda' if torch.cuda.is_available() else 'cpu'  # Use GPU if available
)

# Define parameter distribution for randomized search
param_dist = {
    'module__input_dim': [20],
    'module__hidden_dim1': randint(10, 100),
    'module__hidden_dim2': randint(10, 100),
    'module__output_dim': [2],
    'lr': uniform(0.01, 0.2),
    'optimizer': [optim.SGD, optim.Adam, optim.RMSprop],
    'max_epochs': randint(10, 30)
}

# Perform randomized search with cross-validation
rs = RandomizedSearchCV(net, param_dist, n_iter=20, refit=True, cv=3, scoring='accuracy', verbose=2, n_jobs=-1, random_state=42)
rs.fit(X.astype(np.float32), y)

# Print the best parameters and best score
print("Best parameters found: ", rs.best_params_)
print("Best cross-validation accuracy: ", rs.best_score_)

# Evaluate the best model on the whole dataset
y_pred = rs.predict(X.astype(np.float32))
accuracy = accuracy_score(y, y_pred)
print("Accuracy on the whole dataset: ", accuracy)
